In [16]:
# Following Mike's version but include all the months into one list to work with.

In [1]:
sc

In [2]:
import geopandas as gpd
import pyproj
import shapely.geometry as geom
import csv
import datetime

In [7]:
proj = pyproj.Proj(init='epsg:2263', preserve_units=True)
COLLISIONS_FN = 'collisions_cleaned.csv'
JANUARY_FN = '1january.csv'
FEBRUARY_FN = '2february.csv'
MARCH_FN = '3march.csv'
APRIL_FN = '4april.csv'
MAY_FN = '5may.csv'
JUNE_FN = '6june.csv'

In [11]:
collisions = sc.textFile(COLLISIONS_FN,use_unicode=False).cache()
january = sc.textFile(JANUARY_FN,use_unicode=False).cache()
february = sc.textFile(FEBRUARY_FN,use_unicode=False).cache()
march = sc.textFile(MARCH_FN,use_unicode=False).cache()
april = sc.textFile(APRIL_FN,use_unicode=False).cache()
may = sc.textFile(MAY_FN,use_unicode=False).cache()
june = sc.textFile(JUNE_FN,use_unicode=False).cache()

In [12]:
list(enumerate(collisions.first().split(',')))

[(0, 'DATE'),
 (1, 'TIME'),
 (2, 'BOROUGH'),
 (3, 'ZIP CODE'),
 (4, 'LATITUDE'),
 (5, 'LONGITUDE'),
 (6, 'LOCATION'),
 (7, 'ON STREET NAME'),
 (8, 'CROSS STREET NAME'),
 (9, 'OFF STREET NAME'),
 (10, 'NUMBER OF PERSONS INJURED'),
 (11, 'NUMBER OF PERSONS KILLED'),
 (12, 'NUMBER OF PEDESTRIANS INJURED'),
 (13, 'NUMBER OF PEDESTRIANS KILLED'),
 (14, 'NUMBER OF CYCLIST INJURED'),
 (15, 'NUMBER OF CYCLIST KILLED'),
 (16, 'NUMBER OF MOTORIST INJURED'),
 (17, 'NUMBER OF MOTORIST KILLED'),
 (18, 'CONTRIBUTING FACTOR VEHICLE 1'),
 (19, 'CONTRIBUTING FACTOR VEHICLE 2'),
 (20, 'CONTRIBUTING FACTOR VEHICLE 3'),
 (21, 'CONTRIBUTING FACTOR VEHICLE 4'),
 (22, 'CONTRIBUTING FACTOR VEHICLE 5'),
 (23, 'UNIQUE KEY'),
 (24, 'VEHICLE TYPE CODE 1'),
 (25, 'VEHICLE TYPE CODE 2'),
 (26, 'VEHICLE TYPE CODE 3'),
 (27, 'VEHICLE TYPE CODE 4'),
 (28, 'VEHICLE TYPE CODE 5'),
 (29, 'Counts'),
 (30, 'Collision Rate'),
 (31, 'LAT'),
 (32, 'LONG')]

In [15]:
def extractCyclists(partitionId,partition):
    if partitionId == 0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if row[14]!= '' and row[14] != '0':
#             location = row[6]
            lat = row[6].split(',')[0][1:9] + '' , '' + row[6].split(',')[1][0:9]
#             lng = row[6].split(',')[1][0:9]
            injured = int(row[14]) + int(row[15])    # includes injury and death         
            yield (lat),injured

cyclistAccidents = collisions.mapPartitionsWithIndex(extractCyclists).reduceByKey(lambda x,y: x+y) #.max(lambda x:x[1])
cyclistAccidents.collect()

[(('40.68808', '-73.87668'), 2),
 (('40.67189', '-73.89586'), 6),
 (('40.71792', '-74.00894'), 1),
 (('40.70512', '-73.74659'), 1),
 (('40.71105', '-73.94210'), 2),
 (('40.77246', '-73.96704'), 8),
 (('40.69558', '-73.92076'), 3),
 (('40.71052', '-73.80408'), 1),
 (('40.69832', '-73.94295'), 1),
 (('40.71199', '-74.00810'), 2),
 (('40.58593', '-73.95188'), 2),
 (('40.62757', '-73.97657'), 1),
 (('40.69619', '-73.98876'), 7),
 (('40.79970', '-73.96996'), 1),
 (('40.67854', '-73.94968'), 1),
 (('40.63390', '-73.97045'), 1),
 (('40.74546', '-73.87582'), 1),
 (('40.63347', '-74.02991'), 2),
 (('40.74356', '-73.85575'), 1),
 (('40.68420', '-73.96805'), 3),
 (('40.76122', '-73.93054'), 1),
 (('40.65160', '-73.93329'), 1),
 (('40.70194', '-73.95572'), 3),
 (('40.72103', '-73.90409'), 1),
 (('40.75780', '-73.97352'), 1),
 (('40.70614', '-73.83176'), 1),
 (('40.58612', '-73.95010'), 1),
 (('40.70899', '-73.90970'), 1),
 (('40.62745', '-73.97806'), 2),
 (('40.69093', '-73.92456'), 1),
 (('40.812

In [18]:
list(enumerate(january.first().split(',')))

[(0, 'pickup_datetime'),
 (1, 'pickup_latitude'),
 (2, 'pickup_longitude'),
 (3, 'trip_distance'),
 (4, 'passenger_count'),
 (5, 'dropoff_datetime'),
 (6, 'dropoff_latitude'),
 (7, 'dropoff_longitude'),
 (8, 'fare_amount'),
 (9, 'tolls_amount'),
 (10, 'taxes_amount'),
 (11, 'tip_amount'),
 (12, 'payment_amount'),
 (13, 'payment_type')]

In [19]:
def extractTrips(partitionId,partition):
    if partitionId == 0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
#         if row[1] and row[2] !='0.0' and row[6] and row [7] != '0.0':
        pickup = row[1],row[2]
        dropoff = row[6],row[7]       
        yield (pickup,1)
        yield (dropoff,1)

taxiTrips = january.mapPartitionsWithIndex(extractTrips).reduceByKey(lambda x,y: x+y) #.max(lambda x:x[1])
taxiTrips.take(20)

[(('40.7715', '-73.98176'), 3),
 (('40.75423', '-73.98248'), 2),
 (('40.72778', '-73.9981'), 2),
 (('40.86527', '-73.92799'), 1),
 (('40.72114', '-74.00437'), 3),
 (('40.71324', '-74.00444'), 1),
 (('40.75568', '-73.97303'), 8),
 (('40.763', '-73.96199'), 4),
 (('40.7484', '-73.98248'), 2),
 (('40.72863', '-73.98586'), 4),
 (('40.75374', '-73.97157'), 2),
 (('40.76411', '-73.95389'), 1),
 (('40.76735', '-73.95976'), 1),
 (('40.74197', '-73.99137'), 1),
 (('40.71377', '-74.0085'), 1),
 (('40.78423', '-73.97021'), 8),
 (('40.7883', '-73.97548'), 1),
 (('40.76825', '-73.95634'), 3),
 (('40.70814', '-74.00863'), 2),
 (('40.79848', '-73.9734'), 1)]

In [20]:
taxiTrips.count()

10482803

In [21]:
sorted(cyclistAccidents.join(taxiTrips).take(20))

[(('40.68161', '-73.99851'), (1, 2)),
 (('40.69328', '-73.97385'), (4, 2)),
 (('40.71347', '-74.00481'), (1, 1)),
 (('40.72579', '-73.99761'), (2, 1)),
 (('40.73641', '-74.00194'), (1, 1)),
 (('40.74002', '-74.00529'), (2, 5)),
 (('40.74047', '-73.99828'), (3, 8)),
 (('40.74843', '-73.99619'), (2, 2)),
 (('40.75077', '-73.97653'), (1, 2)),
 (('40.75112', '-73.97127'), (1, 4)),
 (('40.75144', '-73.97395'), (3, 4)),
 (('40.75165', '-73.99013'), (4, 6)),
 (('40.75474', '-73.98789'), (4, 1)),
 (('40.75649', '-73.97236'), (2, 1)),
 (('40.75699', '-73.96698'), (1, 5)),
 (('40.75714', '-73.99356'), (3, 4)),
 (('40.76365', '-73.97346'), (5, 8)),
 (('40.77723', '-73.94616'), (1, 3)),
 (('40.78119', '-73.97974'), (2, 7)),
 (('40.84868', '-73.93921'), (2, 1))]

In [22]:
cyclistAccidents.join(taxiTrips).count()

1306

In [23]:
#taxiTrips.join(cyclistAccidents).collect()

In [24]:
taxiTrips2 = february.mapPartitionsWithIndex(extractTrips).reduceByKey(lambda x,y: x+y) #.max(lambda x:x[1])
taxiTrips2.take(20)

[(('40.69716', '-73.92748'), 1),
 (('40.73458', '-73.99209'), 2),
 (('40.72786', '-73.9992'), 6),
 (('40.77201', '-73.94765'), 2),
 (('40.76609', '-73.98708'), 11),
 (('40.71942', '-73.98958'), 3),
 (('40.76738', '-73.95361'), 1),
 (('40.70621', '-74.01684'), 1),
 (('40.78073', '-73.9526'), 2),
 (('40.85215', '-73.93481'), 1),
 (('40.78048', '-73.97652'), 3),
 (('40.77072', '-73.96582'), 1),
 (('40.74515', '-73.99429'), 2),
 (('40.71365', '-74.01389'), 2),
 (('40.72594', '-74.00766'), 1),
 (('40.71412', '-73.98946'), 1),
 (('40.76006', '-73.97408'), 5),
 (('40.80235', '-73.95883'), 1),
 (('40.77769', '-73.98229'), 2),
 (('40.81186', '-73.96104'), 1)]

In [25]:
taxiTrips2.join(cyclistAccidents).count()

1277

In [26]:
taxiTrips3 = march.mapPartitionsWithIndex(extractTrips).reduceByKey(lambda x,y: x+y) #.max(lambda x:x[1])
taxiTrips3.take(10)

[(('40.75969', '-73.92171'), 1),
 (('40.81625', '-73.95422'), 1),
 (('40.75275', '-73.97994'), 1),
 (('40.7581', '-73.96333'), 1),
 (('40.78034', '-73.95497'), 3),
 (('40.77381', '-73.873'), 1),
 (('40.77565', '-73.9871'), 1),
 (('40.72182', '-74.00636'), 1),
 (('40.75147', '-73.99426'), 2),
 (('40.74751', '-73.99189'), 1)]

In [27]:
taxiTrips3.join(cyclistAccidents).count()

1321

In [29]:
taxiTrips4 = april.mapPartitionsWithIndex(extractTrips).reduceByKey(lambda x,y: x+y) #.max(lambda x:x[1])
taxiTrips4.take(10)

[(('40.75507', '-73.97053'), 1),
 (('40.69518', '-73.99574'), 1),
 (('40.76151', '-73.97553'), 1),
 (('40.73204', '-73.81037'), 1),
 (('40.75953', '-73.97138'), 2),
 (('40.7966', '-73.93629'), 1),
 (('40.68639', '-73.84486'), 1),
 (('40.73456', '-74.00081'), 1),
 (('40.74005', '-73.98243'), 4),
 (('40.78345', '-73.95068'), 2)]

In [30]:
taxiTrips4.join(cyclistAccidents).count()

1320

In [35]:
taxiTrips5 = may.mapPartitionsWithIndex(extractTrips).reduceByKey(lambda x,y: x+y) #.max(lambda x:x[1])
taxiTrips5.take(10)

[(('40.73831', '-73.98162'), 2),
 (('40.75918', '-73.97967'), 1),
 (('40.77641', '-73.98237'), 2),
 (('40.74074', '-73.98159'), 2),
 (('40.73114', '-73.9824'), 3),
 (('40.77304', '-73.9623'), 4),
 (('40.74509', '-73.99342'), 1),
 (('40.64433', '-73.78298'), 1),
 (('40.75837', '-73.96618'), 4),
 (('40.7625', '-73.96576'), 3)]

In [36]:
taxiTrips5.join(cyclistAccidents).count()

1298

In [37]:
taxiTrips6 = june.mapPartitionsWithIndex(extractTrips).reduceByKey(lambda x,y: x+y) #.max(lambda x:x[1])
taxiTrips6.take(10)

[(('40.75089', '-73.99056'), 6),
 (('40.75516', '-73.98074'), 1),
 (('40.78695', '-73.95425'), 2),
 (('40.71033', '-74.01482'), 1),
 (('40.73478', '-73.97984'), 2),
 (('40.78559', '-73.98218'), 1),
 (('40.75131', '-73.971'), 3),
 (('40.77381', '-73.873'), 2),
 (('40.73246', '-73.85104'), 1),
 (('40.76951', '-73.96917'), 2)]

In [38]:
taxiTrips6.join(cyclistAccidents).count()

1327

In [39]:
bike_taxi = taxiTrips.join(cyclistAccidents).collect()
bike_taxi2 = taxiTrips2.join(cyclistAccidents).collect()
bike_taxi3 = taxiTrips3.join(cyclistAccidents).collect()
bike_taxi4 = taxiTrips4.join(cyclistAccidents).collect()
bike_taxi5 = taxiTrips5.join(cyclistAccidents).collect()
bike_taxi6 = taxiTrips6.join(cyclistAccidents).collect()

In [69]:
bike = bike_taxi + bike_taxi2 + bike_taxi3 + bike_taxi4 + bike_taxi5 + bike_taxi6

In [79]:
with open('bike_taxi_full.csv', 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(['LATITUDE AND LONGITUDE', 'TAXITRIP WITH ACCIDENTS'])
    for i in range (0,len(bike)):
        wr.writerow(bike[i])

In [80]:
FULL_LIST = 'bike_taxi_full.csv'
full = sc.textFile(FULL_LIST,use_unicode=False).cache()

In [81]:
print list(enumerate(full.first().split(',')))

[(0, '"LATITUDE AND LONGITUDE"'), (1, '"TAXITRIP WITH ACCIDENTS"')]


In [82]:
def extractFull(partitionId,partition):
    if partitionId == 0:
        partition.next()
    import csv
    reader = csv.reader(partition)
    for row in reader:
        if len(row[0])!= 0:
            latlon = row[0]
            tripsAccidents = row[1]
            yield latlon, tripsAccidents

results = full.mapPartitionsWithIndex(extractFull)
results.take(10)

[("('40.75144', '-73.97395')", '(4, 3)'),
 ("('40.72579', '-73.99761')", '(1, 2)'),
 ("('40.75714', '-73.99356')", '(4, 3)'),
 ("('40.75474', '-73.98789')", '(1, 4)'),
 ("('40.75699', '-73.96698')", '(5, 1)'),
 ("('40.77723', '-73.94616')", '(3, 1)'),
 ("('40.76365', '-73.97346')", '(8, 5)'),
 ("('40.75165', '-73.99013')", '(6, 4)'),
 ("('40.71347', '-74.00481')", '(1, 1)'),
 ("('40.73641', '-74.00194')", '(1, 1)')]